In [1]:
#!pip install geopy

In [1]:
import pandas as pd
from geopy.distance import geodesic
import re

In [2]:
# 讀取事故資料作為df_a
df_a = pd.read_csv('C:/專題/交通事故合併天氣科技執法測站ID/台南市_A2_new_20230705.csv');df_a

# 讀取測站資料作為df_b
df_b = pd.read_csv('C:/專題/交通事故合併天氣科技執法測站ID/氣象測站清單20230607.csv');df_b

# 讀取測速資料作為df_c
df_c = pd.read_csv('C:/專題/交通事故合併天氣科技執法測站ID/臺南市_camera.csv');df_c

C:\Users\student\AppData\Local\Temp\ipykernel_8848\2671162053.py:2: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df_a = pd.read_csv('C:/專題/交通事故合併天氣科技執法測站ID/台南市_A2_new_20230705.csv');df_a


,SOURCE,TEMP ID,縣市,種類,行政區,轄區分局,地點,行向,經度,緯度,取締項目(RAW),取締項目(CLEAN),超速速限,區間測速偵測長度,未依標誌標線行駛,闖紅燈,超速,未禮讓行人,限制車種違規駛入,違規停車
0,人工,TN0024,臺南市,固定式交通違規照相路段,北門區,學甲分局,臺61線281.9公里,南向,120.137872,23.292835,超速,超速,90,NaN,0,0,1,0,0,0
1,人工,TN0001,臺南市,固定式交通違規照相路段,新營區,新營分局,臺1線290.3公里與東山路口,北向,120.325730,23.308907,超速,超速,50,NaN,0,0,1,0,0,0
2,人工,TN0002,臺南市,固定式交通違規照相路段,新營區,新營分局,長榮路、三興街口172線20.05公里,東向,120.300354,23.319435,超速,超速,50,NaN,0,0,1,0,0,0
3,人工,TN0003,臺南市,固定式交通違規照相路段,新營區,新營分局,南314線1公里,雙向,120.335900,23.309875,超速,超速,50,NaN,0,0,1,0,0,0
4,人工,TN0004,臺南市,固定式交通違規照相路段,新營區,新營分局,長榮路一段與東泰路口,西向,120.326116,23.320008,超速,超速,60,NaN,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,人工,TN0131,臺南市,違規停車科技執法,北區,第五分局,北區臺南轉運站前公園路周邊路段,雙向,120.209300,23.002090,違規停車、違規臨時停車、違規占用機(慢)車優先道臨時停車等,違規停車,50,NaN,0,0,0,0,0,1
131,程式,TN0132,臺南市,違規停車科技執法,永康區,永康分局,中華路與中華一路口周邊路段(近兵仔市場)\n【中華路東側自門牌96號至204號、中華路西側自...,雙向,120.235248,23.004374,違規停車、違規臨時停車、違規占用機(慢)車優先道臨時停車等,違規停車,50,NaN,0,0,0,0,0,1
132,人工,TN0133,臺南市,違規停車科技執法,新化區,新化分局,中正路與信義路路口周邊路段,多向,120.308300,23.034960,違規停車,違規停車,50,NaN,0,0,0,0,0,1
133,人工,TN0134,臺南市,區間平均速率科技執法,龍崎區,歸仁分局,市道182線22.1公里至28公里處(往關廟),雙向,120.388094,22.951212,超速,超速,50,"5,870.2公尺",0,0,1,0,0,0


In [4]:
# 將測速資料的經緯度轉為數值
df_c['緯度'] = pd.to_numeric(df_c['緯度'], errors='coerce')
df_c['經度'] = pd.to_numeric(df_c['經度'], errors='coerce')

# 將經緯度為缺失值的那筆資料刪掉
df_c = df_c.dropna(subset=['緯度', '經度'])
df_c.isna().sum()

SOURCE           0
TEMP ID          0
縣市               0
種類               0
行政區              0
轄區分局             0
地點               0
行向               0
經度               0
緯度               0
取締項目(RAW)        0
取締項目(CLEAN)      0
超速速限             0
區間測速偵測長度       133
未依標誌標線行駛         0
闖紅燈              0
超速               0
未禮讓行人            0
限制車種違規駛入         0
違規停車             0
dtype: int64

In [5]:
# 日期的正規表達式
patternYears_month_date = r"(\d+)-(\d+)-(\d+)"

In [6]:
# 將日期的年月日分別切割之後放在list裡
def spilt_date(date):
    date_list = []
    match = re.search(patternYears_month_date, date)
    date_list.append(match.group(1))
    date_list.append(match.group(2))
    date_list.append(match.group(3))
    return date_list

# 該方法用來判斷該場車禍是否在指定測站的營運時間內，a、b、c值分別為六都車禍的日期、測站開始日期、測站撤銷日期
def during_activation_True_False(a, b, c):
    try:
        accident_date = spilt_date(a)
        station_start_date = spilt_date(b)
    
    except AttributeError:
        return_ture_false= False
        return return_ture_false
    
    if station_start_date[0]>accident_date[0]:
        after_start_date = False
        
    elif station_start_date[0] == accident_date[0]:
        
        if station_start_date[1]>accident_date[1]:
            after_start_date = False
            
        elif station_start_date[1] == accident_date[1]:
            
            if station_start_date[2] > accident_date[2]:
                after_start_date = False
                
            else:
                after_start_date = True
        else:
            after_start_date = True
                
    else:
        after_start_date = True
    
    if after_start_date:
        
        if pd.notna(c):
            station_stop_date = spilt_date(c)
            
            if accident_date[0] > station_stop_date[0]:
                return_ture_false = False
            
            elif accident_date[0] == station_stop_date[0]:
                
                if accident_date[1] > station_stop_date[1]:
                    return_ture_false = False
                    
                elif accident_date[1] == station_stop_date[1]:
                    
                    if accident_date[2] > station_stop_date[2]:
                        return_ture_false = False
                        
                    else: 
                        return_ture_false = True
                    
                else:
                    return_ture_false = True
                
            else: 
                return_ture_false = True
        
        else:
            return_ture_false= True
        
    else:
        return_ture_false= False
        
    return return_ture_false

# 對於df_a表中進來的每一筆資料，找到最接近的測站
def find_nearest_station():
    #變數初始化為正無窮大，用於追蹤最小的距離
    min_distance = float('inf')
    #變數初始化為 None，用於存儲最近車站的站號
    station_id = None
    for _, coord in df_b.iterrows():
        distance = geodesic((row['LATITUDE'], row['LONGITUDE']), (coord['緯度'], coord['經度'])).kilometers
        if distance < min_distance:
            if during_activation_True_False(row['WHOLE_DATE'], coord['資料起始日期'], coord['撤站日期']):
                min_distance = distance
                station_id = coord['站號']
            else:
                continue
    return station_id

# # 對於df_a表中進來的每一筆資料，找到最接近的測速，並限定範圍
def find_nearest_camera():
    min_distance = float('inf')
    camera_id = None
    limit_distance = 0.3
    for _, coord in df_c.iterrows():
        distance = geodesic((row['LATITUDE'], row['LONGITUDE']), (coord['緯度'], coord['經度'])).kilometers
        if distance < min_distance:
            min_distance = distance
            camera_id = coord['TEMP ID']
    if min_distance <= limit_distance:
        return camera_id

    else:
        return None

    

In [79]:
# # 建立空列
# new_column = pd.Series(dtype='str')

# # 在指定位置插入空的列
# df_a.insert(51, 'STATION_ID', new_column)
# df_a.insert(52, 'CAMERA_ID', new_column)

In [7]:
# 執行一次就好(如果不小心重複執行，就看下面的程式執行到哪筆再把0修改為看到的最後一筆的數字)
stop_point = 484420

In [8]:
# 上面遇到未知狀況停止，除錯完之後再執行這個程式
for index, row in df_a.iterrows():
    if index >= stop_point:
        df_a.loc[index, 'STATION_ID'] = find_nearest_station()
        df_a.loc[index, 'CAMERA_ID'] = find_nearest_camera()
        if index%1000 ==0:
            df_a.to_csv('台南市_A2_new_20230705.csv', index=False, encoding= 'UTF-8-sig')
        stop_point += 1
        print(f'第{index}筆已新增完畢')

第484420筆已新增完畢
第484421筆已新增完畢
第484422筆已新增完畢
第484423筆已新增完畢
第484424筆已新增完畢
第484425筆已新增完畢
第484426筆已新增完畢
第484427筆已新增完畢
第484428筆已新增完畢
第484429筆已新增完畢
第484430筆已新增完畢
第484431筆已新增完畢
第484432筆已新增完畢
第484433筆已新增完畢
第484434筆已新增完畢
第484435筆已新增完畢
第484436筆已新增完畢
第484437筆已新增完畢
第484438筆已新增完畢
第484439筆已新增完畢
第484440筆已新增完畢
第484441筆已新增完畢
第484442筆已新增完畢
第484443筆已新增完畢
第484444筆已新增完畢
第484445筆已新增完畢
第484446筆已新增完畢
第484447筆已新增完畢
第484448筆已新增完畢
第484449筆已新增完畢
第484450筆已新增完畢
第484451筆已新增完畢
第484452筆已新增完畢
第484453筆已新增完畢
第484454筆已新增完畢
第484455筆已新增完畢
第484456筆已新增完畢
第484457筆已新增完畢
第484458筆已新增完畢
第484459筆已新增完畢
第484460筆已新增完畢
第484461筆已新增完畢
第484462筆已新增完畢
第484463筆已新增完畢
第484464筆已新增完畢
第484465筆已新增完畢
第484466筆已新增完畢
第484467筆已新增完畢
第484468筆已新增完畢
第484469筆已新增完畢
第484470筆已新增完畢
第484471筆已新增完畢
第484472筆已新增完畢
第484473筆已新增完畢
第484474筆已新增完畢
第484475筆已新增完畢
第484476筆已新增完畢
第484477筆已新增完畢
第484478筆已新增完畢
第484479筆已新增完畢
第484480筆已新增完畢
第484481筆已新增完畢
第484482筆已新增完畢
第484483筆已新增完畢
第484484筆已新增完畢
第484485筆已新增完畢
第484486筆已新增完畢
第484487筆已新增完畢
第484488筆已新增完畢
第484489筆已新增完畢
第484490筆已新增完畢
第48449

In [9]:
# 儲存結果到新的CSV檔案(檔名命名:縣市名稱+事故資料新增測站ID和測速ID+當下日期.csv)
df_a.to_csv('台南市_A2_new_20230705.csv', index=False, encoding= 'UTF-8-sig')

In [9]:
stop_point